# Лабораторная работа 2

Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода между 2013 и 2015 годами.
На загруженных данных обучите преобразование PCA с числом компонент равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?
Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.
Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?
Какая компания имеет наибольший вес в первой компоненте? Укажите ее название.

### Часть 1. Загрузка данных, применение РСА к данным (количество компонент 10) 

In [2]:
import pandas as pd
import numpy as np

#Чтение данных из файла
price = pd.read_csv('close_prices.csv',delimiter=',')

#Преобразование данных в массив NumPy
x = price.to_numpy()
x = x[:,1:31].astype(float)

#Метод главных компонет
xc = x - np.mean(x , axis = 0)   #Центрирование данных
xn = xc/np.std(xc , axis = 0)   #Нормирование данных
cov_mat = np.cov(xn , rowvar = False)   #Расчет матрицы ковариации
eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)   #Находим собственные значения и векторы матрицы ковариации
sorted_index = np.argsort(eigen_values)[::-1]   #Сортируем по убыванию собственные значения
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:,sorted_index]   #Сортируем по убыванию собственные векторы
n = 10   #Требуемое количество компонент
eigenvector_subset = sorted_eigenvectors[:,0:n]   #Выбираем собственные векторы в количестве n
xr = np.dot(eigenvector_subset.transpose() , xn.transpose()).transpose()   #Компактная форма
xn_re = np.dot(xr, eigenvector_subset.transpose())   #Восстановление данных

### Часть 2. Сколько компонент хватит, чтобы объяснить 90% дисперсии?

Предполагаемый ответ. Если сложить все 30 sorted_eigenvalue, то получится некое число (30.08042895442359), которое означает 100% дисперсии. Следовательно нужно найти такое количество sorted_eigenvalue, которое в сумме даст число, большее (30.08042895442359*0.9 = 27,07238606)

In [4]:
perc = np.zeros(30, float)
for i in range(30):
    perc[i] = sorted_eigenvalue[i] + perc[i-1]
perc = perc/perc[29]
print('Количество компонент, чтобы объяснить 90% дисперсии равно:', np.nonzero(perc > 0.9)[0][0] + 1)

Количество компонент, чтобы объяснить 90% дисперсии равно: 6


### Часть 3. Возьмите значение первой компоненты

In [5]:
first = xr[:,0]

### Часть 4. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv

In [6]:
#Чтение данных из файла
djia = pd.read_csv('djia_index.csv',delimiter=',')

#Преобразование данных в массив NumPy
dd = djia.to_numpy()
dd = dd[:,1].astype(float)
ddc = dd - np.mean(dd , axis = 0)   #Центрирование данных
ddn = ddc/np.std(ddc , axis = 0)   #Нормирование данных

### Часть 6. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

In [7]:
print('Корреляция Пирсона между первой компонентой и индексом Доу-Джонса равна:', np.corrcoef(first, ddn)[0, 1])

Корреляция Пирсона между первой компонентой и индексом Доу-Джонса равна: -0.9468481520945369


### Часть 7. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название.

In [19]:
corr = np.zeros(30, float)
for i in range(30):
    corr[i] = np.absolute(np.corrcoef(xn[:,i], first)[0, 1])

col_names = pd.read_csv('close_prices.csv', nrows=0).columns[1:]
print('Название компании, имеющей наибольший вес в первой компоненте:', col_names[np.argmax(corr)])

Название компании, имеющей наибольший вес в первой компоненте: MMM
